In [1]:
import tensorflow as tf
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:

def extract_fileImg(list_paths,batch_val=0,batch_test=0):
    
    labels=[]
    label_names=[]
    batch=2000
    
    image_files = tf.io.gfile.glob(list_paths)
    
    for img_path in image_files:
        if img_path.split('.')[1].split('/')[2] == "cat":
            labels.append(1)
            label_names.append("cat")
        else :
            labels.append(0)
            label_names.append("dog")
    
    if batch_val > 0:
        image_files = image_files[batch:batch+batch_val]
        labels = labels[batch:batch+batch_val]
        label_names[batch:batch+batch_val]
    if batch_test > 0:
        batch_val = 100
        image_files = image_files[batch+batch_val:batch+batch_val+batch_test]
        labels = labels[batch+batch_val:batch+batch_val+batch_test]
        label_names[batch+batch_val:batch+batch_val+batch_test]
    else:
        image_files = image_files[:batch]
        labels = labels[:batch]
        label_names = label_names[:batch]
            
    return image_files,labels,label_names

In [3]:
def preprocess_image(img_path):
    img_data = tf.io.read_file(img_path)
    feat = tf.image.decode_jpeg(img_data,channels=3)
    feat = tf.image.convert_image_dtype(feat,tf.float32)
    feat = tf.image.resize(feat,(128,128))
    return feat
    

In [4]:
# Processus DataSet-TRAIN
file_pattern = ["./train/*.jpg"]
image_files,label_train,label_name_train = extract_fileImg(file_pattern)
image_train = np.array([preprocess_image(i).numpy() for i in image_files])
dataset_train = tf.data.Dataset.from_tensor_slices((image_train,tf.cast(label_train,tf.float32))).shuffle(2000).batch(500)
dataset_train


<BatchDataset shapes: ((None, 128, 128, 3), (None,)), types: (tf.float32, tf.float32)>

In [5]:
# Processus DataSet-VAL
file_pattern = ["./train/*.jpg"]
image_files,label_val,label_name_val = extract_fileImg(file_pattern,batch_val=100)
image_val = np.array([preprocess_image(i).numpy() for i in image_files])
dataset_val = tf.data.Dataset.from_tensor_slices((image_val,tf.cast(label_val,tf.float32))).shuffle(100).batch(100)
dataset_val

<BatchDataset shapes: ((None, 128, 128, 3), (None,)), types: (tf.float32, tf.float32)>

In [6]:
# Processus DataSet-TEST
file_pattern = ["./train/*.jpg"]
image_files,label_test,label_name_test= extract_fileImg(file_pattern,batch_test=100)
image_test = np.array([preprocess_image(i).numpy() for i in image_files])
dataset_test = tf.data.Dataset.from_tensor_slices((image_test,tf.cast(label_test,tf.float32))).shuffle(100).batch(100)
dataset_test

<BatchDataset shapes: ((None, 128, 128, 3), (None,)), types: (tf.float32, tf.float32)>

In [7]:
model = tf.keras.models.Sequential([

    #block 1
    tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu',padding='same',input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    #block 2
    tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    #Dense layers:
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(10,activation='relu'),
    #tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation="softmax")
                                
])

model.compile(loss= tf.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              metrics=["accuracy"])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 61504)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                3936320   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6

In [ ]:
callbacks=[
    tf.keras.callbacks.EarlyStopping(patience=100, monitor='val_loss'), 
    #tf.keras.callbacks.TensorBoard(log_dir='/home/malloc/logs/fit1', write_graph=True)
           ]

model.fit(dataset_train, epochs=5, callbacks=callbacks,validation_data=dataset_val)

Epoch 1/5


In [ ]:

model.evaluate(dataset_test)


In [ ]:

y_predict = model.predict(dataset_test)
size=24
rows=8
cols=4
fig = plt.figure(figsize=(15,20))
for i,index in enumerate(np.random.choice(image_test.shape[0],size=size,replace=False)):
    axis = fig.add_subplot(rows,cols,i+1,xticks=[],yticks=[])
    axis.imshow(image_test[index])
    index_p = np.argmax(y_predict[index])
    index_t = int(label_test[index])
    axis.set_title(("{},({})").format(label_name_test[index_p],label_name_test[index_p]), color=("green" if index_p==index_t else "red"))
    
